In [45]:
import boto3
import os

s3 = boto3.resource('s3')

def download_s3_folder(bucket_name: str, s3_folder: str, local_dir: str =None):
    """
    Download the contents of a folder directory
    Args:
        bucket_name: the name of the s3 bucket
        s3_folder: the folder path in the s3 bucket
        local_dir: a relative or absolute directory path in the local file system
    """
    bucket = s3.Bucket(bucket_name)
    for obj in bucket.objects.filter(Prefix=s3_folder):
        target = obj.key if local_dir is None \
            else os.path.join(local_dir, os.path.relpath(obj.key, s3_folder))
        if not os.path.exists(os.path.dirname(target)):
            os.makedirs(os.path.dirname(target))
        if obj.key[-1] == '/':
            continue
        bucket.download_file(obj.key, target)

download_s3_folder(
    bucket_name='iot-workshop-publish-ip',
    s3_folder='topic/publish_ip',
    local_dir='C:\\Users\\heiko\\OneDrive\\02_Arbeit\\2021-_DONE\\05_Conferences\\2022_CompanyWorkshop\\01_IoT\\repo\\ip_addresses\\'
)

In [46]:
import pandas as pd
import os

# Read the files in the directory
path = 'C:\\Users\\heiko\\OneDrive\\02_Arbeit\\2021-_DONE\\05_Conferences\\2022_CompanyWorkshop\\01_IoT\\repo\\ip_addresses\\'
files = os.listdir(path)
dfs = [pd.read_json(f'{path}{f}', typ='series') for f in files]
# pd.read_json('C:\\Users\\heiko\\OneDrive\\02_Arbeit\\2021-_DONE\\05_Conferences\\2022_CompanyWorkshop\\01_IoT\\repo\\ip_addresses\\1665424686895.json', typ='series')
df = pd.concat(dfs, axis=1).T
df['Timestamp'] = pd.to_datetime(df['Timestamp'], infer_datetime_format=True)

df.sort_values(by='Timestamp').drop_duplicates(subset=["username"], keep="last")

,Timestamp,username,ip
53,2022-10-10 08:51:06,done2,192.168.1.115
59,2022-10-12 03:41:05,done3,192.168.1.117
